In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import category_encoders as ce

# pandasの行を省略しない
pd.set_option('display.max_columns', None)

In [40]:
categorical_columns = ['curb_loc', 'steward', 'guards',
       'sidewalk', 'user_type', 'problems', 'spc_common', 'spc_latin', 'nta',
       'nta_name', 'borocode', 'boro_ct', 'boroname', 'zip_city', 'cb_num',
       'st_senate', 'st_assem', 'cncldist']

In [41]:
train_df = pd.read_csv('C:/python/signate/data/train.csv', index_col=0)
test_df = pd.read_csv('C:/python/signate/data/test.csv', index_col=0)
sample_df = pd.read_csv('C:/python/signate/data/sample_submission.csv', header=None, index_col=0)

train_df.head()


,created_at,tree_dbh,curb_loc,health,steward,guards,sidewalk,user_type,problems,spc_common,spc_latin,nta,nta_name,borocode,boro_ct,boroname,zip_city,cb_num,st_senate,st_assem,cncldist
0,2015-06-29,14,OnCurb,1,NaN,NaN,Damage,Volunteer,NaN,English oak,Quercus robur,QN45,Douglas Manor-Douglaston-Little Neck,4,4152901,Queens,Little Neck,411,11,25,23
1,2016-09-21,5,OnCurb,1,3or4,Helpful,NoDamage,Volunteer,NaN,crimson king maple,Acer platanoides 'Crimson King',BX05,Bedford Park-Fordham North,2,2039901,Bronx,Bronx,207,33,78,15
2,2015-09-13,26,OnCurb,2,NaN,NaN,NoDamage,Volunteer,StonesBranchLights,English oak,Quercus robur,SI01,Annadale-Huguenot-Prince's Bay-Eltingville,5,5017011,Staten Island,Staten Island,503,24,62,51
3,2016-05-09,15,OnCurb,0,NaN,NaN,Damage,NYC Parks Staff,NaN,honeylocust,Gleditsia triacanthos var. inermis,SI11,Charleston-Richmond Valley-Tottenville,5,5024401,Staten Island,Staten Island,503,24,62,51
4,2016-06-24,23,OnCurb,1,NaN,NaN,NoDamage,Volunteer,Stones,London planetree,Platanus x acerifolia,MN03,Central Harlem North-Polo Grounds,1,1022102,Manhattan,New York,110,30,70,9


データはほとんどカテゴリカル
#### 思いついた精度向上案
- commonを個別名称として、latinの上を属名として利用する
- 地区が多すぎるので整理する

In [42]:
# train_dfのcolumnを取得
columns = test_df.columns

In [43]:
def cleansing(df):
    df.fillna("None", inplace=True)

    # 日付を月情報だけにする
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['created_at'] = df['created_at'].dt.month
    return df

train_clean = cleansing(train_df)
test_clean = cleansing(test_df)

In [44]:
train_clean.head()

,created_at,tree_dbh,curb_loc,health,steward,guards,sidewalk,user_type,problems,spc_common,spc_latin,nta,nta_name,borocode,boro_ct,boroname,zip_city,cb_num,st_senate,st_assem,cncldist
0,6,14,OnCurb,1,None,None,Damage,Volunteer,None,English oak,Quercus robur,QN45,Douglas Manor-Douglaston-Little Neck,4,4152901,Queens,Little Neck,411,11,25,23
1,9,5,OnCurb,1,3or4,Helpful,NoDamage,Volunteer,None,crimson king maple,Acer platanoides 'Crimson King',BX05,Bedford Park-Fordham North,2,2039901,Bronx,Bronx,207,33,78,15
2,9,26,OnCurb,2,None,None,NoDamage,Volunteer,StonesBranchLights,English oak,Quercus robur,SI01,Annadale-Huguenot-Prince's Bay-Eltingville,5,5017011,Staten Island,Staten Island,503,24,62,51
3,5,15,OnCurb,0,None,None,Damage,NYC Parks Staff,None,honeylocust,Gleditsia triacanthos var. inermis,SI11,Charleston-Richmond Valley-Tottenville,5,5024401,Staten Island,Staten Island,503,24,62,51
4,6,23,OnCurb,1,None,None,NoDamage,Volunteer,Stones,London planetree,Platanus x acerifolia,MN03,Central Harlem North-Polo Grounds,1,1022102,Manhattan,New York,110,30,70,9


In [49]:
train_df['curb_loc'].unique()

array(['OnCurb', 'OffsetFromCurb'], dtype=object)

In [45]:
# 一旦lightgbmで学習してみる
# count encodingとlabel encodingを行う
ce_columns =  ['curb_loc', 'steward', 'guards',
       'sidewalk', 'user_type', 'problems', 'spc_common', 'spc_latin', 'nta',
       'nta_name', 'borocode', 'boro_ct', 'boroname', 'zip_city', 'cb_num',
       'st_senate', 'st_assem', 'cncldist']
le_columns = ['created_at', 'curb_loc', 'steward', 'guards',
       'sidewalk', 'user_type', 'problems', 'spc_common', 'spc_latin', 'nta',
       'nta_name', 'borocode', 'boro_ct', 'boroname', 'zip_city', 'cb_num',
       'st_senate', 'st_assem', 'cncldist']

y = train_df['health']
train_clean_drop = train_clean.drop(['health'], axis=1)
test_clean_drop = test_clean

ce = ce.CountEncoder(cols=categorical_columns)
ce.fit(train_clean_drop)

train_clean_drop = ce.transform(train_clean_drop)
test_clean_drop = ce.transform(test_clean_drop)

# trainとvalidに分割
X_train, X_test, y_train, y_test = train_test_split(train_clean_drop, y, test_size=0.2, random_state=0)
print(len(X_train))

# LightGBM用のデータセットに変換
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# LightGBMのハイパーパラメータの設定
params = {
  'objective':'multiclass', # 目的 : 多クラス分類
  'metric':'multi_logloss', # 評価指標 : 不正解率(= 1-正解率)
  'num_class':3,        # クラス数 : 3
  'verbosity':-1,       # 学習の状況を表示しない

}

# lightgbmインスタンス生成
model = lgb.LGBMClassifier(**params)

# 学習
model.fit(X_train, y_train, eval_metric='multi_logloss', 
           eval_set=[(X_test, y_test)], )

# テストデータでの予測
y_pred = model.predict(X_test)
# Calculate accuracy score
print("Accuracy:", f1_score(y_test, y_pred, average='macro'))



15987
Accuracy: 0.29887308510959515


'\n# LightGBMモデルの学習\nmodel = lgb.train(params, train_data,\n                   num_boost_round=1000, valid_sets=[train_data, test_data], \n                    categorical_feature=categorical_columns, \n                    )\n\n# テストデータでの予測\ny_pred = model.predict(X_test)\ny_pred_max = np.argmax(y_pred, axis=1)\n# Calculate accuracy score\nprint("Accuracy:", f1_score(y_test, y_pred_max, average=\'macro\'))\n'

In [ ]:
y_pred = model.predict(test_clean_drop)
sample_df[1] = y_pred
sample_df.to_csv('C:/python/signate/data/base2.csv', header=None)

In [48]:
# 特徴量の重要度の算出（データフレーム）
pd.Series(model.feature_importances_,index=X_train.columns).to_frame(name='重要度')

,重要度
created_at,646
tree_dbh,1147
curb_loc,53
steward,191
guards,212
sidewalk,196
user_type,219
problems,625
spc_common,1122
spc_latin,0
